In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
datasetDir = 'Dataset'

# get the Subdirectory of the Dataset Folder
for root, folder, files in os.walk(datasetDir):
    if folder != []:
        listDir = folder

classes = []
# retrieve the name and the number of the classification
for i, category in enumerate(listDir):
    # print(category)
    classes.append(category)
    # classes.append({'name' : category,'id' : i + 1, "category" : category})
classes

['Bag',
 'Book',
 'Chair',
 'Cup',
 'Eraser',
 'Fork',
 'Pen',
 'Pencil',
 'Pencil Case',
 'Plate',
 'Soap',
 'Spoon',
 'Table',
 'Toothbrush',
 'Toothpaste']

In [3]:
from keras.preprocessing.image import ImageDataGenerator

# Create a datagen that is split the dataset into 80% Train and 20% Test
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    # shear_range=0.2,
    zoom_range=0.5,
    width_shift_range=0.5,
    height_shift_range=0.5,
    horizontal_flip=True,
    validation_split=0.2,
)


test_datagen = ImageDataGenerator(
    rescale=1./255,
)

# Training Set
training_set = datagen.flow_from_directory(
    datasetDir,
    target_size=(299,299),
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    subset="training",
    batch_size=32
)

training_set_2 = datagen.flow_from_directory(
    datasetDir,
    target_size=(224,224),
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    subset="training",
    batch_size=32
)

# Validation Set
validation_set = datagen.flow_from_directory(
    datasetDir,
    target_size=(299,299),
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    subset="validation",
    batch_size=32
)

validation_set_2 = datagen.flow_from_directory(
    datasetDir,
    target_size=(224,224),
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    subset="validation",
    batch_size=32
)

test_set = test_datagen.flow_from_directory(
    'Test Set',
    target_size=(299,299),
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    batch_size=32
)

test_set_2 = test_datagen.flow_from_directory(
    'Test Set',
    target_size=(224,224),
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    batch_size=32
)

Found 2615 images belonging to 15 classes.
Found 2615 images belonging to 15 classes.
Found 647 images belonging to 15 classes.
Found 647 images belonging to 15 classes.
Found 291 images belonging to 15 classes.
Found 291 images belonging to 15 classes.


In [4]:
# image_path = 'Dataset/Soap/Soap_4.jpg'
# augment = 'vertical_flip'

# img = tf.keras.preprocessing.image.load_img(image_path, target_size= (224,224))
# img_tensor = tf.keras.preprocessing.image.img_to_array(img)
# img_tensor = np.expand_dims(img_tensor, axis=0)

# #Uses ImageDataGenerator to flip the images
# show_datagen = ImageDataGenerator(
#     # rescale=1./255,
#     # rotation_range=90,
#     # shear_range=20,
#     # zoom_range=0.5,
#     # width_shift_range=[-0.2, 0.2],
#     # height_shift_range=[-0.2, 0.2],
#     # horizontal_flip=True,
#     vertical_flip=True,
#     # validation_split=0.2,
# )

# #Creates our batch of one image
# pic = show_datagen.flow(img_tensor, batch_size =1)
# fig = plt.figure(figsize=(10,8))
# img_flat = np.squeeze(img_tensor)
# ax1 = fig.add_subplot(1,5,1)
# ax1.title.set_text('Original')
# plt.imshow((img_flat).astype(np.uint8))
# plt.axis('off')

# #Plots our figures
# for i in range(2,5):
#   fig.add_subplot(1, 5, i)
#   batch = pic.next()
#   image_ = batch[0].astype('uint8')
#   plt.imshow(image_)
#   plt.axis('off')
#   plt.savefig(f"Augment/{augment}.png", bbox_inches='tight')
# plt.show()


In [59]:
InceptionV3 = tf.keras.models.load_model('Model/InceptionV3/InceptionV3-Adam-0.0001-256Batch.h5')
InceptionV3.evaluate(test_set, verbose=1,  return_dict=True)

10/10 [==============================] - 6s 382ms/step - loss: 0.6529 - accuracy: 0.8282


{'loss': 0.6529161930084229, 'accuracy': 0.8281787037849426}

In [60]:
Vgg_19 = tf.keras.models.load_model('Model/VGG-19/VGG-19-Adam-0.0001-256Batch.h5')
Vgg_19.evaluate(test_set_2 , verbose=1,  return_dict=True)

10/10 [==============================] - 5s 421ms/step - loss: 1.5738 - accuracy: 0.6598


{'loss': 1.573818564414978, 'accuracy': 0.6597937941551208}

In [61]:
MobileNet = tf.keras.models.load_model('Model/MobileNet/MobileNet-Nadam-0.0001-256Batch.h5')
MobileNet.evaluate(test_set_2, verbose=1,return_dict=True)

10/10 [==============================] - 4s 277ms/step - loss: 1.6557 - accuracy: 0.7285


{'loss': 1.6556593179702759, 'accuracy': 0.7285223603248596}

In [8]:
for layer in Vgg_19.layers:
    layer._name = layer.name + str('_2')
    layer.trainable = False

for layer in MobileNet.layers:
    layer.trainable = False

for layer in InceptionV3.layers:
    layer.trainable = False

In [9]:
# merged =  tf.keras.layers.Concatenate(axis=1)([MobileNet.layers[-1].output, Vgg_19.layers[-1].output])
# classify = tf.keras.layers.Dense(len(classes) , activation='softmax')(merged)
# model = tf.keras.Model(inputs=[(MobileNet.input, Vgg_19.input)] , outputs=classify)

In [10]:
# model.compile(
#     optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics= ['accuracy']
# )


In [11]:
# model.fit(
#     [training_set_2 , training_set_2], 
#     epochs= 10,
#     verbose = 1,
#     validation_data = [validation_set_2, validation_set_2]
# )